# ImagingStack Workflow

This notebook demonstrates how to create, inspect, downsample, and save imaging volumes using the `ImagingStack` helper class.

## 1. Imports

We import NumPy for generating sample data and bring in the `ImagingStack` class.

In [ ]:
import os
from pathlib import Path
import numpy as np
from ImagingStack import ImagingStack

## 2. Create a Synthetic Volume

In lieu of a real imaging file, we will generate a 3D NumPy array and attach simple metadata (voxel sizes, etc.).

In [ ]:
volume = np.random.rand(30, 64, 64).astype(np.float32)
metadata = {
    'voxel_size': (2.0, 0.8, 0.8),
    'description': 'Synthetic volume for docs example',
    'channels': ['GFP'],
}
volume.shape, volume.dtype

## 3. Build an ImagingStack

Instantiate the class with the data and metadata. Printing the object shows the summary from `__repr__`.

In [ ]:
stack = ImagingStack(volume, metadata=metadata)
print(stack)

### Inspect metadata
You can read or modify metadata directly since it is stored as a regular dictionary.

In [ ]:
stack.metadata

## 4. Downsample the stack

Use `downsample` with either a scalar scale factor or a 3-tuple for anisotropic reductions.

In [ ]:
half_stack = stack.downsample(scale=0.5, method='nearest')
print(half_stack)
half_stack.metadata.get('voxel_size')

## 5. Save to disk

Choose a format by filename extension or pass `fmt`. This example writes an NRRD file to the working directory.

In [ ]:
output_path = 'synthetic_stack.nrrd'
half_stack.to_file(output_path, fmt='nrrd')
output_path

## 6. Load an existing file

When you have a real `.nrrd` or `.nii/.nii.gz` file, use `from_file` to build an `ImagingStack`. Replace the path below with your own data.

In [ ]:
# example_path = 'path/to/your_image.nii.gz'
# loaded_stack = ImagingStack.from_file(example_path)
# print(loaded_stack)

## 7. Real brain example

Load the provided NRRD file, downsample it by 0.5, and save the result as a NIfTI file on the Desktop. Adjust the paths if your data lives elsewhere.

In [ ]:
real_brain_path = os.path.expanduser("~/Desktop/ImagingStack/PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_01.nrrd")
real_output_path = os.path.expanduser("~/Desktop/ImagingStack/PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_01_downsampled.nii.gz")

real_stack = ImagingStack.from_file(real_brain_path)
print(real_stack)

real_ds = real_stack.downsample(scale=0.5, method='nearest')
print(real_ds)

real_ds.to_file(real_output_path, fmt='nifti')
real_output_path

## 8. ND2 workflow example

Demonstrates loading the sample ND2 file, downsampling, and writing channel 1 as a compressed NIfTI.

In [ ]:
from pathlib import Path
data_dir = Path('/Users/erikduboue/Desktop/ImagingStack')
nd2_path = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005.nd2'
nd2_stack = ImagingStack.from_file(str(nd2_path))
print(nd2_stack)
nd2_ds = nd2_stack.downsample(scale=0.5, method='nearest')
nd2_base_output = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nd2_downsampled.nii.gz'
saved_paths = []
for ch in (1, 2, 3):
    try:
        nd2_ds.to_file(str(nd2_base_output), fmt='nifti', channel=ch)
        saved_paths.append(data_dir / f"PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nd2_downsampled_{ch:02d}.nii.gz")
    except ValueError:
        saved_paths.append(None)
saved_paths


## 9. NRRD workflow example

Load the provided NRRD file, downsample, and save channel 1.

In [ ]:
nrrd_path = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005.nrrd'
nrrd_stack = ImagingStack.from_file(str(nrrd_path))
print(nrrd_stack)
nrrd_ds = nrrd_stack.downsample(scale=(0.5, 0.5, 1.0), method='nearest')
nrrd_output = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nrrd_downsampled.nii.gz'
nrrd_ds.to_file(str(nrrd_output), fmt='nifti', channel=1)
nrrd_saved = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nrrd_downsampled_01.nii.gz'
nrrd_saved

## 10. NIfTI workflow example

Load the provided NIfTI file, downsample, and save channel 1.

In [ ]:
nii_path = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005.nii'
nii_stack = ImagingStack.from_file(str(nii_path))
print(nii_stack)
nii_ds = nii_stack.downsample(scale=0.5, method='nearest')
nii_output = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nii_downsampled.nii.gz'
nii_ds.to_file(str(nii_output), fmt='nifti', channel=1)
nii_saved = data_dir / 'PA1_6dpf_gcamp488_slc1a3b546_sox10647_005_nii_downsampled_01.nii.gz'
nii_saved